## Fetch Data from API (SerpAPI)

In [2]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import requests
import os

In [102]:
tokens = {
    'technology': "CAAqJggKIiBDQkFTRWdvSUwyMHZNRGRqTVhZU0FtVnVHZ0pWVXlnQVAB",
    'sports': 'CAAqJggKIiBDQkFTRWdvSUwyMHZNRFp1ZEdvU0FtVnVHZ0pWVXlnQVAB',
    'entertainment': 'CAAqJggKIiBDQkFTRWdvSUwyMHZNREpxYW5RU0FtVnVHZ0pWVXlnQVAB',
}

In [56]:
def fetch_data(topic_token, section_token=None):
    url = "https://serpapi.com/search"

    params = {
    'engine': 'google_news',
    'hl': 'en',
    'topic_token': topic_token,
    'api_key': os.environ.get("API_KEY")
    }

    if section_token:
        params['section_token'] = section_token

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
        else:
            data = {}
            response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except requests.exceptions.RequestException as e:
        print(f"Other error occurred: {e}")

    return data

In [115]:
def extract_data(topic_token):
    titles = []
    section_tokens = [i['section_token'] for i in fetch_data(topic_token)['sub_menu_links']]

    for section_token in section_tokens:
        data = fetch_data(topic_token, section_token)['news_results']
        for item in data:
            title = item.get('title', item.get('highlight', {}).get('title', None))
            stories = item.get('stories', [])
            titles.extend([title] + [story.get('title', None) for story in stories])

    return titles

In [116]:
data = extract_data(tokens['entertainment'])